https://saturncloud.io/blog/how-to-serve-tensorflow-models-in-amazon-sagemaker/

In [22]:
import numpy as np
import tarfile

In [16]:
train_np =np.random.uniform(size=(5 ,5))
test_np =np.random.uniform(size=(5,5))

In [17]:
train_labels = np.array([0, 1, 1, 0, 1])
test_labels = np.array([1, 0, 1, 1, 0])


In [19]:
# import libraries
import tensorflow as tf
from tensorflow import keras

# Create a simple model
model = keras.Sequential([
    keras.layers.Dense(10, activation='relu', input_shape=(5, )),
    keras.layers.Dense(1, activation='sigmoid')
])


# Compile the model
model.compile(optimizer='adam',
              loss = "binary_crossentropy",
              metrics=['accuracy'])

# Train the model

EPOCHS = 50
BATCH_SIZE = 32

EARLY_STOPPING = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", mode="auto", restore_best_weights=True
)

history = model.fit(
    x=train_np,
    y=train_labels,
    validation_data=(test_np, test_labels),
    callbacks=[EARLY_STOPPING],
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
)


Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 0.6704 - accuracy: 0.8000 - val_loss: 0.7210 - val_accuracy: 0.4000
Epoch 2/50
1/1 [==============================] - 0s 29ms/step - loss: 0.6693 - accuracy: 0.8000 - val_loss: 0.7205 - val_accuracy: 0.4000
Epoch 3/50
1/1 [==============================] - 0s 29ms/step - loss: 0.6683 - accuracy: 0.8000 - val_loss: 0.7200 - val_accuracy: 0.6000
Epoch 4/50
1/1 [==============================] - 0s 30ms/step - loss: 0.6672 - accuracy: 0.8000 - val_loss: 0.7196 - val_accuracy: 0.6000
Epoch 5/50
1/1 [==============================] - 0s 30ms/step - loss: 0.6661 - accuracy: 0.8000 - val_loss: 0.7192 - val_accuracy: 0.6000
Epoch 6/50
1/1 [==============================] - 0s 32ms/step - loss: 0.6650 - accuracy: 0.8000 - val_loss: 0.7188 - val_accuracy: 0.6000
Epoch 7/50
1/1 [==============================] - 0s 30ms/step - loss: 0.6640 - accuracy: 0.8000 - val_loss: 0.7184 - val_accuracy: 0.6000
Epoch 8/50
1/1 [=============

In [23]:
model.save("export/Servo/1")
with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("export")


INFO:tensorflow:Assets written to: export/Servo/1/assets


INFO:tensorflow:Assets written to: export/Servo/1/assets


In [24]:
bucket_name="model-255423"

In [26]:
from sagemaker import get_execution_role
from sagemaker import Session
role = get_execution_role()
sm_session = Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [27]:
s3_response = sm_session.upload_data("model.tar.gz", bucket=bucket_name, key_prefix="model")



In [29]:
tf.__version__

'2.16.2'

In [28]:
from sagemaker.tensorflow import TensorFlowModel

# Select which type of SageMaker EC2 instance to deploy the model on  
instance_type = 'ml.m5.xlarge' 

In [30]:
sagemaker_model = TensorFlowModel(
    model_data=f"s3://{bucket_name}/model/model.tar.gz",
    role=role,
    framework_version=tf.__version__,
)


In [31]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type=instance_type)

-----!CPU times: user 338 ms, sys: 36.2 ms, total: 374 ms
Wall time: 3min 2s


In [32]:
sample = test_np

In [33]:
predictor.predict(sample)

{'predictions': [[0.54835546],
  [0.480862081],
  [0.511126161],
  [0.423835307],
  [0.548755169]]}